In [1]:
!pip install -q datasets
!pip install -q torchtext==0.17.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 800.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.modules.activation import LeakyReLU
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data import get_tokenizer
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader

import numpy as np
import math
import string
import re

BUILD TRANSFORMER

In [3]:
class TokenAndPositionEmbedding(nn.Module):
  def __init__(self, vocab_size, embedding_dim, max_len, device = 'cpu'):
    super().__init__()
    self.device = device;
    self.word_embedding = nn.Embedding(
        num_embeddings= vocab_size,
        embedding_dim = embedding_dim
    )
    self.position_embedding = nn.Embedding(
        num_embeddings= max_len,
        embedding_dim = embedding_dim
    )

  def forward(self,x):
    N_samples_in_batch, seq_length = x.size()
    positions = torch.arange(0, seq_length).expand(N_samples_in_batch, seq_length).to(self.device)
    output1 = self.word_embedding(x)
    output2 = self.position_embedding(positions)
    return output1 + output2

In [4]:
class TransformerEncoderBlock(nn.Module):
  def __init__(self, embedding_dim, num_heads, ffn_dim, dropout = 0.1):
    super().__init__ ()
    self.attention = nn.MultiheadAttention(
        embed_dim= embedding_dim,
        num_heads= num_heads,
        batch_first= True  # B x Sqen_len x Em_dim
    )
    self.cross_attention = nn.MultiheadAttention(
        embed_dim= embedding_dim,
        num_heads= num_heads,
        batch_first= True
    )
    self.ffn = nn.Sequential(
        nn.Linear(in_features= embedding_dim, out_features= ffn_dim,bias= True),
        nn.LeakyReLU(),
        nn.Linear(in_features= ffn_dim, out_features= embedding_dim, bias= True)
    )
    self.layerNorm = nn.LayerNorm(normalized_shape= embedding_dim, eps= 1e-06)
    self.dropout = nn.Dropout(p= dropout)

  def forward(self, q, k, v):
    attention_output, _ = self.attention(query= q, key= k, value= v)
    attention_output = self.dropout(attention_output)

    output_1 = self.layerNorm(q + attention_output) # attention output + q (skip connection)

    ffn_output = self.ffn(output_1)
    ffn_output = self.dropout(ffn_output)

    output_2 = self.layerNorm(output_1 + ffn_output) # ffn output + output_1 (skip connection)

    return output_2

In [5]:
class TransformerEncoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim, max_length, num_layers, num_heads, ffn_dim, dropout = 0.1, device = 'cpu'):
    super().__init__()
    self.embedding = TokenAndPositionEmbedding(
        vocab_size= vocab_size,
        embedding_dim= embedding_dim,
        max_len= max_length,
        device= device
    )
    self.layers = nn.ModuleList(
        [
            TransformerEncoderBlock(
                embedding_dim= embedding_dim,
                num_heads= num_heads,
                ffn_dim= ffn_dim,
                dropout= dropout
            ) for _ in range(num_layers)
        ]
    )

  def forward(self,x):
    output = self.embedding(x)
    for layer in self.layers:
      output = layer(output,output,output)
      return output

In [6]:
#test
batch_size = 32
vocal_size = 1000
embedding_dim = 200
max_length = 100
num_layers = 3
num_heads = 4
ffn_dim = 1028

input = torch.randint(
    high= 2,
    size= (batch_size, max_length), #32x100
    dtype= torch.int64
)


In [7]:
encoder = TransformerEncoder(
    vocab_size= vocal_size,
    embedding_dim= embedding_dim,
    max_length= max_length,
    num_layers= num_layers,
    num_heads= num_heads,
    ffn_dim= ffn_dim
)

encoded = encoder(input)
encoded.shape

torch.Size([32, 100, 200])

In [8]:
class TransformerDecoderBlock(nn.Module):
  def __init__(self, embedding_dim, num_heads, ffn_dim, dropout= 0.1):
    super().__init__()
    self.attention = nn.MultiheadAttention(
        embed_dim= embedding_dim,
        num_heads= num_heads,
        batch_first= True
    )
    self.cross_attention = nn.MultiheadAttention(
        embed_dim= embedding_dim,
        num_heads= num_heads,
        batch_first= True
    )
    self.ffn = nn.Sequential(
        nn.Linear(
            in_features=embedding_dim,
            out_features= ffn_dim,
            bias= True
            ),
        nn.LeakyReLU(),
        nn.Linear(
            in_features=ffn_dim,
            out_features= embedding_dim,
            bias= True
            ),
    )
    self.layerNorm = nn.LayerNorm(normalized_shape= embedding_dim, eps= 1e-06)
    self.dropout = nn.Dropout(p= dropout)

  def forward(self, x, encoder_output, src_mask, tgt_mask):
    attention_output, _ = self.attention(query= x, key= x, value= x, attn_mask= tgt_mask)
    attention_output = self.dropout(attention_output)
    output_1 = self.layerNorm(x + attention_output)

    cross_attention_output, _ = self.cross_attention(query= output_1, key= encoder_output, value= encoder_output, attn_mask= src_mask)
    cross_attention_output = self.dropout(cross_attention_output)
    output_2 = self.layerNorm(output_1 + cross_attention_output)

    ffn_output = self.ffn(output_2)
    ffn_output = self.dropout(ffn_output)

    output_3 = self.layerNorm(output_2 + ffn_output)

    return output_3

In [9]:
class TransformerDecoder(nn.Module):
  def __init__(self, vocal_size, embedding_dim, max_length, num_layers, num_heads, ffn_dim, dropout = 0.1, device = 'cpu'):
    super().__init__()
    self.embedding = TokenAndPositionEmbedding(
        vocab_size= vocal_size,
        embedding_dim= embedding_dim,
        max_len= max_length,
        device= device
    )
    self.layers = nn.ModuleList([
        TransformerDecoderBlock(
            embedding_dim= embedding_dim,
            num_heads= num_heads,
            ffn_dim= ffn_dim,
            dropout= dropout
        ) for _ in range(num_layers)
    ])
  def forward(self, x, encoder_output, src_mask, tgt_mask):
    output = self.embedding(x)
    for layer in self.layers:
      output = layer(output, encoder_output, src_mask, tgt_mask)

    return output

In [10]:
class Transformer(nn.Module):
  def __init__(self, src_vocab_size, tgt_vocab_size,
               embedding_dim, max_length, num_layers, num_heads, ffn_dim,
               dropout = 0.1, device = 'cpu'):
    super().__init__()
    self.device = device
    self.encoder = TransformerEncoder(
        vocab_size= src_vocab_size,
        embedding_dim= embedding_dim,
        max_length= max_length,
        num_layers= num_layers,
        num_heads= num_heads,
        ffn_dim= ffn_dim,
        dropout= dropout,
        device= device
    )
    self.decoder = TransformerDecoder(
        vocal_size= tgt_vocab_size,
        embedding_dim= embedding_dim,
        max_length= max_length,
        num_layers= num_layers,
        num_heads= num_heads,
        ffn_dim= ffn_dim,
        dropout= dropout,
        device= device
    )
    self.fc = nn.Linear(in_features= embedding_dim, out_features= tgt_vocab_size)

  def generate_mask(self, src, tgt):
    src_seq_len = src.shape[1]
    tgt_seq_len = tgt.shape[1]

    src_mask = torch.zeros(
        size= (src_seq_len, src_seq_len),
        device= self.device
    ).type(torch.bool)

    tgt_mask = (
        torch.triu(torch.ones(size= (tgt_seq_len, tgt_seq_len),device= self.device)) == 1
        ).transpose(0,1)
    tgt_mask = tgt_mask.float().masked_fill(tgt_mask == 0, float('-inf')).masked_fill(tgt_mask == 1, float(0.0))

    return src_mask, tgt_mask

  def forward(self,src, tgt):
    src_mask, tgt_mask = self.generate_mask(src, tgt)
    encoder_output = self.encoder(src)
    decoder_output = self.decoder(tgt, encoder_output, src_mask, tgt_mask)
    output = self.fc(decoder_output)

    return output

In [11]:
src_seq_len = 3
src_mask = torch.zeros(
        size= (src_seq_len, src_seq_len),
    ).type(torch.bool)
src_mask

tensor([[False, False, False],
        [False, False, False],
        [False, False, False]])

In [12]:
tgt_seq_len = 3
tgt_mask = torch.ones(size= (tgt_seq_len, tgt_seq_len))
print(tgt_mask)

tgt_mask = torch.triu(tgt_mask)
print(tgt_mask)

tgt_mask = tgt_mask == 1
print(tgt_mask)

tgt_mask = tgt_mask.transpose(0,1)
print(tgt_mask)


tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[1., 1., 1.],
        [0., 1., 1.],
        [0., 0., 1.]])
tensor([[ True,  True,  True],
        [False,  True,  True],
        [False, False,  True]])
tensor([[ True, False, False],
        [ True,  True, False],
        [ True,  True,  True]])


In [13]:
#test
batch_size = 128
src_vocab_size = 1000
tgt_vocab_size = 2000
embed_dim = 200
max_length = 100
num_layers = 2
num_heads = 4
ff_dim = 256

model = Transformer(
    src_vocab_size , tgt_vocab_size ,
    embed_dim , max_length , num_layers , num_heads , ff_dim
)

src = torch.randint(
    high =2,
    size =(batch_size , max_length),
    dtype = torch.int64
)

tgt = torch.randint(
    high =2,
    size =(batch_size, max_length),
    dtype = torch.int64
    )
prediction = model (src, tgt)
prediction.shape # batch_size x max_length x tgt_vocab_size

torch.Size([128, 100, 2000])

TEST WITH TEXT CLASSIFICATION

In [14]:
from datasets import load_dataset

ds = load_dataset("thainq107/ntc-scv")
ds

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/570 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.8M [00:00<?, ?B/s]

data/valid-00000-of-00001.parquet:   0%|          | 0.00/6.35M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/6.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'preprocessed_sentence'],
        num_rows: 30000
    })
    valid: Dataset({
        features: ['sentence', 'label', 'preprocessed_sentence'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['sentence', 'label', 'preprocessed_sentence'],
        num_rows: 10000
    })
})

In [15]:
#preprocessing
def preprocessing_text(text):
  #remove url web
  url_pattern = re.compile(r'https ?://\ s+\ wwww \.\s+')
  text = url_pattern.sub(r" ", text)

  #remove html tag
  html_pattern = re.compile(r'<[^ < >]+ >')
  text = html_pattern.sub(" ", text)

  #remove punctuations and digits
  replace_chars = list(string.punctuation + string.digits)
  for char in replace_chars:
    text = text.replace(char, " ")

  #remove emoji
  emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F" # emoticons
      u"\U0001F300-\U0001F5FF" # symbols & pictographs
      u"\U0001F680-\U0001F6FF" # transport & map symbols
      u"\U0001F1E0-\U0001F1FF" # flags (iOS)
      u"\U0001F1F2-\U0001F1F4" # Macau flag
      u"\U0001F1E6-\U0001F1FF" # flags
      u"\U0001F600-\U0001F64F"
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      u"\U0001f926-\U0001f937"
      u"\U0001F1F2"
      u"\U0001F1F4"
      u"\U0001F620"
      u"\u200d"
      u"\u2640-\u2642"
      "]+", flags = re.UNICODE
  )
  text = emoji_pattern.sub(r" ", text)

  #normalize whitespace
  text = " ".join(text.split())

  #lowercase
  text = text.lower()

  return text

In [16]:
def yield_tokens(sentences, tokenizer):
  for sentence in sentences:
    yield tokenizer(sentence)

#tokenizer
tokenizer = get_tokenizer("basic_english")

#build vocab
vocab_size = 10000
vocab = build_vocab_from_iterator(
    yield_tokens(ds['train']['preprocessed_sentence'], tokenizer),
    specials = ['<pad>','<unk>'],
    max_tokens = vocab_size
)

vocab.set_default_index(vocab['<unk>'])

#convert dataset
def prepare_dataset(data):
  #create iter (sentence, label)
  for row in data:
    sentence = row["preprocessed_sentence"]
    encoded_senctence = vocab(tokenizer(sentence))
    label = row["label"]
    yield encoded_senctence, label

train_dataset = prepare_dataset(ds['train'])
train_dataset = to_map_style_dataset(train_dataset)

test_dataset = prepare_dataset(ds['test'])
test_dataset = to_map_style_dataset(test_dataset)

val_dataset = prepare_dataset(ds['valid'])
val_dataset = to_map_style_dataset(val_dataset)

In [19]:
#dataloader
seq_length = 100
barch_size = 128

def collate_fn(batch):
  sentences, labels = zip(*batch)
  encoded_sentences = [
      sentence + ([0] * (seq_length - len(sentence))) if len(sentence) < seq_length # [0] idx cua <pad>
      else sentence[:seq_length]
      for sentence in sentences
  ]

  encoded_sentences = torch.tensor(encoded_sentences, dtype= torch.int64)
  labels = torch.tensor(labels)

  return encoded_sentences, labels

train_dataloader = DataLoader(
    train_dataset,
    batch_size= barch_size,
    shuffle= True,
    collate_fn= collate_fn
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size= barch_size,
    shuffle= False,
    collate_fn= collate_fn
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size= barch_size,
    shuffle= False,
    collate_fn= collate_fn
)

In [ ]:
def train(args):
